# Prompt Template으로 배포한 AI Service 사용하기
Prompt Template을 사용하여 AI Service를 배포한 후, 해당 서비스를 사용하는 방법에 대해 설명합니다.


In [1]:
from dotenv import load_dotenv
import os
# .env 파일 로드
load_dotenv()
# IBM Cloud API Key와 URL 설정
ibm_cloud_url = os.environ['IBM_CLOUD_URL']
project_id=os.environ['PROJECT_ID']
api_key = os.environ['API_KEY']

## 환경정보 설정

In [2]:
# 배포한 프롬프트 템플릿 모델의 AI Service URL
ibm_ai_service_url = "https://us-south.ml.cloud.ibm.com/ml/v1/deployments/18d4a2e6-add0-4215-a0cb-c67ab4130f90/text/generation?version=2021-05-01"

번역할 원문을 입력합니다.

In [3]:
user_input = "3일동안 저녁에 타우린 먹어야해"

배포한 AI Service를 호출하여 처리 결과를 확인합니다.

In [4]:
import requests
import json

# IBM Cloud API Key를 사용하여 토큰 발급
token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={
        "apikey": api_key,
        "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
    }
)
mltoken = token_response.json().get("access_token")
if not mltoken:
    raise Exception("토큰 발급 실패!")

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {mltoken}',
    'Accept': 'application/json'  # JSON 응답
}

payload_scoring = {
    "parameters": 
        {
            "prompt_variables": 
            {
                "default" : user_input
            }
        }
    
}

response = requests.post(
    ibm_ai_service_url,
    headers=headers,
    json=payload_scoring,
    stream=False
)


print("=== 응답 결과 ===")
if response.status_code == 200:
    result = response.json()
    print(result)
    
    # generated_text 추출
    if 'results' in result and len(result['results']) > 0:
        generated_text = result['results'][0]['generated_text']
        first_line = generated_text.split('\n\nInput')[0]
        
        print("input : " + user_input)
        print("Output :" + first_line)
    else:
        print("응답에서 결과를 찾을 수 없습니다.")
else:
    print(f"오류 발생: {response.status_code}")
    print(response.text)


=== 응답 결과 ===
{'model_id': 'mistralai/mistral-large', 'model_version': '2.0.0', 'created_at': '2025-08-05T01:00:55.579Z', 'results': [{'generated_text': '                     {\n  "medications": [\n    {\n      "name": "타우린",\n      "frequency": 1,\n      "times": ["18:00"],\n      "original_text": "3일동안 저녁에 타우린 먹어야해"\n    }\n  ],\n  "calendar_events": [\n    {\n      "summary": "💊 타우린 복용",\n      "description": "약물: 타우린\\n복용 시간: 18:00\\n⚠️ 정확한 시간에 복용하세요!",\n      "start": {\n        "dateTime": "2025-08-03T18:00:00+09:00",\n        "timeZone": "Asia/Seoul"\n      },\n      "end": {\n        "dateTime": "2025-08-03T18:30:00+09:00",\n        "timeZone": "Asia/Seoul"\n      },\n      "recurrence": [\n        "RRULE:FREQ=DAILY;COUNT=3"\n      ],\n      "reminders": {\n        "useDefault": false,\n        "overrides": [\n          {"method": "popup", "minutes": 10},\n          {"method": "popup", "minutes": 0}\n        ]\n      }\n    }\n  ],\n  "schedule_info": {\n    "start_date": "2025